In [1]:
import zipfile
import os
import time 
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import warnings
warnings.simplefilter('ignore')

### Load Dataset

In [2]:
# Path to the ZIP file containing the training data
df_data = pd.read_csv('datasets/train_set.csv')
df_data.head(10)

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1,1,3,2005,2005,K,K.78,K.78.2,D,C,...,1.008912,0.261040,0.907793,-0.077998,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
1,2,2,2,2005,2003,Q,Q.22,Q.22.3,B,C,...,1.240851,0.432987,-0.726459,0.204785,O,-0.231530,-0.266117,-0.272337,-0.251419,0.0
2,3,3,1,2005,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
3,4,3,1,2006,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
4,5,3,2,2005,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
5,6,3,2,2006,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.231530,-0.266117,-0.272337,-0.251419,0.0
6,7,4,1,2006,2001,AJ,AJ.129,AJ.129.3,G,C,...,0.580718,0.551128,0.416289,-0.024395,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
7,8,4,2,2006,2002,AQ,AQ.17,AQ.17.1,B,C,...,0.527193,-0.023200,-0.701884,0.226663,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
8,9,4,3,2005,2002,AQ,AQ.17,AQ.17.1,B,C,...,0.527193,-0.023200,-0.701884,0.226663,M,-0.231530,-0.266117,-0.272337,-0.251419,0.0
9,10,5,1,2005,1995,BW,BW.3,BW.3.0,D,?,...,0.176312,0.283264,0.969232,-0.792339,N,2.054683,-0.266117,-0.272337,-0.251419,0.0


In [3]:
print(f'Shape of Training data: {df_data.shape}')
print(df_data.columns)

Shape of Training data: (13184290, 35)
Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6',
       'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4',
       'Claim_Amount'],
      dtype='object')


In [4]:
# summary statistic of dataset
df_data.describe()

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
count,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07,1.318429e+07
mean,6.592146e+06,4.128242e+06,1.894002e+00,2.006052e+03,1.999312e+03,-1.011925e-02,-6.508702e-02,-2.543391e-02,-5.456792e-02,3.838594e-03,-4.012271e-02,-2.421288e-02,-5.856059e-02,1.468410e-02,1.751169e-02,1.354226e-02,1.851376e-02,1.360658e+00
std,3.805977e+06,2.248415e+06,1.173861e+00,8.123720e-01,5.211866e+00,9.800609e-01,9.684165e-01,1.018902e+00,9.680170e-01,9.910490e-01,9.792078e-01,1.006433e+00,1.003954e+00,1.031040e+00,1.038212e+00,1.027748e+00,1.034274e+00,3.900103e+01
min,1.000000e+00,1.000000e+00,1.000000e+00,2.005000e+03,1.981000e+03,-2.578222e+00,-2.493393e+00,-2.790335e+00,-2.508216e+00,-3.350344e+00,-2.376657e+00,-2.778491e+00,-2.163042e+00,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
25%,3.296073e+06,2.184932e+06,1.000000e+00,2.005000e+03,1.996000e+03,-6.658971e-01,-8.161519e-01,-8.696874e-01,-7.830189e-01,-6.860239e-01,-6.887650e-01,-8.984857e-01,-6.517680e-01,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
50%,6.592146e+06,4.257083e+06,2.000000e+00,2.006000e+03,2.000000e+03,-3.123581e-01,-1.245062e-01,-2.217581e-01,-1.064709e-01,-1.150981e-01,-2.372568e-01,-4.684193e-01,-2.568567e-01,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
75%,9.888218e+06,6.281433e+06,2.000000e+00,2.007000e+03,2.003000e+03,4.429298e-01,4.806838e-01,7.269956e-01,4.855086e-01,5.331405e-01,4.973212e-01,8.217801e-01,3.409799e-01,-2.315299e-01,-2.661168e-01,-2.723372e-01,-2.514189e-01,0.000000e+00
max,1.318429e+07,7.542113e+06,2.900000e+01,2.007000e+03,2.009000e+03,5.143392e+00,7.829420e+00,5.563325e+00,7.589263e+00,4.018167e+00,4.584289e+00,4.127148e+00,4.735074e+01,6.627110e+00,8.883081e+00,8.691144e+00,6.388802e+00,1.144075e+04


### Preprocessing Data

In [5]:
# check for missing values 
df_data.isnull().sum()

Row_ID                0
Household_ID          0
Vehicle               0
Calendar_Year         0
Model_Year            0
Blind_Make            0
Blind_Model           0
Blind_Submodel        0
Cat1                  0
Cat2                  0
Cat3                  0
Cat4                  0
Cat5                  0
Cat6                  0
Cat7                  0
Cat8                  0
Cat9                  0
Cat10                 0
Cat11                 0
Cat12             28882
OrdCat                0
Var1                  0
Var2                  0
Var3                  0
Var4                  0
Var5                  0
Var6                  0
Var7                  0
Var8                  0
NVCat                 0
NVVar1                0
NVVar2                0
NVVar3                0
NVVar4                0
Claim_Amount          0
dtype: int64

In [6]:
# We can see that columns 'Cat12' have 28882 missing values, then we can remove these lines 
df_dropped = df_data.dropna(subset=['Cat12'])
df_dropped.shape

(13155408, 35)

In [7]:
# At this time, our dataset contains columns with text values and columns with numerical values.
# So, we need determine text values columns and encode them to numerical values 

# Determine text value columns 
text_columns = df_dropped.select_dtypes(include=['object']).columns
print(text_columns)

Index(['Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat2', 'Cat3',
       'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Cat8', 'Cat9', 'Cat10', 'Cat11',
       'Cat12', 'OrdCat', 'NVCat'],
      dtype='object')


In [8]:
# Encode to numerical values 
label_encoders = {}
for col in text_columns:
    le = LabelEncoder()
    df_dropped[col] = le.fit_transform(df_dropped[col].astype(str))
    label_encoders[col] = le
df_dropped.head(10)

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1,1,3,2005,2005,60,1029,2040,4,3,...,1.008912,0.261040,0.907793,-0.077998,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
1,2,2,2,2005,2003,66,1141,2263,2,3,...,1.240851,0.432987,-0.726459,0.204785,14,-0.231530,-0.266117,-0.272337,-0.251419,0.0
2,3,3,1,2005,1998,18,369,645,2,0,...,-0.971487,-1.405797,-0.837048,-1.176858,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
3,4,3,1,2006,1998,18,369,645,2,0,...,-0.971487,-1.405797,-0.837048,-1.176858,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
4,5,3,2,2005,2001,53,917,1760,10,3,...,0.812656,2.112691,1.534462,2.347260,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
5,6,3,2,2006,2001,53,917,1760,10,3,...,0.812656,2.112691,1.534462,2.347260,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
6,7,4,1,2006,2001,10,166,252,7,3,...,0.580718,0.551128,0.416289,-0.024395,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
7,8,4,2,2006,2002,17,324,564,2,3,...,0.527193,-0.023200,-0.701884,0.226663,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
8,9,4,3,2005,2002,17,324,564,2,3,...,0.527193,-0.023200,-0.701884,0.226663,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
9,10,5,1,2005,1995,47,820,1597,4,0,...,0.176312,0.283264,0.969232,-0.792339,13,2.054683,-0.266117,-0.272337,-0.251419,0.0


In [9]:
# Next, we remove columns 'RowID' , 'Calendar_Year', 'Model_Year' because they are unneccessary for training 
df_train = df_dropped.drop(columns=['Row_ID', 'Calendar_Year', 'Model_Year'])
df_train.head(10)

,Household_ID,Vehicle,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,Cat3,Cat4,Cat5,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1,3,60,1029,2040,4,3,6,0,0,...,1.008912,0.261040,0.907793,-0.077998,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
1,2,2,66,1141,2263,2,3,1,1,1,...,1.240851,0.432987,-0.726459,0.204785,14,-0.231530,-0.266117,-0.272337,-0.251419,0.0
2,3,1,18,369,645,2,0,1,1,1,...,-0.971487,-1.405797,-0.837048,-1.176858,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
3,3,1,18,369,645,2,0,1,1,1,...,-0.971487,-1.405797,-0.837048,-1.176858,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
4,3,2,53,917,1760,10,3,2,0,0,...,0.812656,2.112691,1.534462,2.347260,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
5,3,2,53,917,1760,10,3,2,0,0,...,0.812656,2.112691,1.534462,2.347260,5,-0.231530,-0.266117,-0.272337,-0.251419,0.0
6,4,1,10,166,252,7,3,1,1,3,...,0.580718,0.551128,0.416289,-0.024395,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
7,4,2,17,324,564,2,3,2,1,1,...,0.527193,-0.023200,-0.701884,0.226663,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
8,4,3,17,324,564,2,3,2,1,1,...,0.527193,-0.023200,-0.701884,0.226663,12,-0.231530,-0.266117,-0.272337,-0.251419,0.0
9,5,1,47,820,1597,4,0,5,0,0,...,0.176312,0.283264,0.969232,-0.792339,13,2.054683,-0.266117,-0.272337,-0.251419,0.0


### Train Test Split

In [10]:
# Seperate features and target variables
X = df_train.drop(columns=['Claim_Amount'])
y = df_train['Claim_Amount']

In [11]:
# train-test-split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
# Transform data 
scaler = StandardScaler()
X_train[X_train.select_dtypes(include=['float64', 'int64']).columns] = scaler.fit_transform(X_train.select_dtypes(include=['float64', 'int64']))
X_test[X_test.select_dtypes(include=['float64', 'int64']).columns] = scaler.transform(X_test.select_dtypes(include=['float64', 'int64']))

In [13]:
print(X_train.head())
print(X_test.head())

         Household_ID   Vehicle  Blind_Make  Blind_Model  Blind_Submodel  \
9113228      0.835029 -0.761609   -0.398455    -0.452342       -0.455564   
5848934     -0.055140  0.942790   -1.144885    -1.083292       -1.084691   
5856419     -0.053967  0.090591   -0.178917    -0.281675       -0.274780   
7679516      0.341432  0.942790   -0.925347    -0.829878       -0.873777   
5893265     -0.048386  0.942790    0.962681     0.985396        0.923213   

             Cat1      Cat2      Cat3      Cat4      Cat5  ...      Var4  \
9113228  1.488091  1.063131  2.520346 -0.883644 -0.916517  ...  0.732678   
5848934 -0.966491 -1.123698  1.242432  0.191392  0.398941  ... -0.403002   
5856419  0.786782 -0.394755 -0.674440  0.191392  3.029856  ... -0.140922   
7679516 -0.966491 -1.123698 -0.674440  0.191392  0.398941  ... -0.979578   
5893265  1.838745  1.063131 -0.035483 -0.883644 -0.916517  ...  0.732678   

             Var5      Var6      Var7      Var8     NVCat    NVVar1    NVVar2  \
91132

### XGBoost Regressor Training

In [14]:
params = {
    'objective':'reg:squarederror', 
    'colsample_bytree': 0.3, 
    'learning_rate': 0.1,
    'max_depth': 10, 
    'alpha':10, 
    'n_estimators': 1000
}

# Initialize XGBoost Regressor model 
xgb_reg = xgb.XGBRegressor(**params)

training_start = time.perf_counter() 

# Training 
xgb_reg.fit(X_train, y_train) 


training_end = time.perf_counter() 
xgb_training_time = training_end - training_start
print(f'Time consumed for training: {xgb_training_time}')

Time consumed for training: 207.54574995900475


### Prediction

In [15]:
y_pred = xgb_reg.predict(X_test)

In [16]:
xgb_rmse = mean_squared_error(y_test, y_pred, squared=False) 
xgb_r2_socre = r2_score(y_test, y_pred)

print(f'Mean Square Error when used XGBoost Regressor: {xgb_rmse}')
print(f'R2 Score when used XGBoost Regressor: {xgb_r2_socre}')

Mean Square Error when used XGBoost Regressor: 38.48506757595425
R2 Score when used XGBoost Regressor: -0.01688787615871523
